In [5]:
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision
from sklearn.svm import SVC
import matplotlib.pyplot as plt


In [6]:
import joblib

svm = joblib.load("svm_model.pkl")

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.efficientnet_b0(weights="DEFAULT")
model.classifier = nn.Identity()  # remove final FC layer
model = model.to(device)
model.eval()

# 3️⃣ Transform to preprocess the camera frames same as training
SVMtransform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# 4️⃣ Start live camera
cap = cv2.VideoCapture(0)  # 0 = default camera

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 5️⃣ Preprocess and extract features
    img_tensor = SVMtransform(frame).unsqueeze(0).to(device)
    with torch.no_grad():
        feats = model(img_tensor).cpu().numpy()
    
    # 6️⃣ Predict using SVM
    pred = svm.predict(feats)[0]

    # 7️⃣ Show prediction on frame
    cv2.putText(frame, f"Prediction: {pred}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Live Camera", frame)

    # 8️⃣ Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
